#Melbourne Calculator time for financial markets
---
This programme computes the opening and closing times of financial markets around the world, based on the time in Melbourne, Australia. The programme generates a six-column DataFrame. 

The first column is titled 'Market' and contains the name of the financial market. 

The second and third columns are labelled 'UTC Open' and 'UTC Close', respectively, and represent the market's opening and closing times in Coordinated Universal Time (UTC). 

The fourth and fifth columns, titled 'Melbourne Open' and 'Melbourne Close,' represent the market's opening and closing times in Melbourne, Australia. These columns are derived by converting the UTC start time to Melbourne time. 

The sixth column is labelled 'Status,' and it indicates whether the market is open or closed at the time in Melbourne. The Status column displays 'Open' if the current time is between the market's opening and closing times. Otherwise, it displays 'Closed'. 

This programme is beneficial for anyone who wishes to keep track of when financial markets throughout the world open and close, especially if they live in Melbourne or work in the Australian financial market.





In [6]:
from datetime import datetime
import pandas as pd
import pytz


# Create a DataFrame with the market information
df = pd.DataFrame({
    'Market': ['New York Stock Exchange', 'London Stock Exchange', 'Tokyo Stock Exchange', 'Shanghai Stock Exchange', 'Hong Kong Stock Exchange', 'Sydney Stock Exchange'],
    'UTC Open': ['14:30', '08:00', '23:00', '01:30', '01:30', '00:00'],
    'UTC Close': ['21:00', '16:30', '08:00', '07:00', '09:00', '06:00']
})

# Convert UTC time to Melbourne time
melbourne_tz = 'Australia/Melbourne'
df['Melbourne Open'] = pd.to_datetime(df['UTC Open']).dt.tz_localize('UTC').dt.tz_convert(melbourne_tz).dt.time
df['Melbourne Close'] = pd.to_datetime(df['UTC Close']).dt.tz_localize('UTC').dt.tz_convert(melbourne_tz).dt.time

# Add a new column with the opening time in minutes from midnight
df['Melbourne Open Minutes'] = df['Melbourne Open'].apply(lambda x: x.hour * 60 + x.minute)


# Add a new column that tells us if the market is open or closed based on the current time in Melbourne
current_time = datetime.now(pytz.timezone(melbourne_tz)).time()
df['Status'] = df.apply(lambda row: 'Open' if (row['Melbourne Open'] <= current_time < row['Melbourne Close']) or (row['Market'] == 'London Stock Exchange' and datetime.now(pytz.timezone('UTC')).time() >= datetime.strptime('08:00', '%H:%M').time() and current_time >= datetime.strptime('07:00', '%H:%M').time()) else 'Closed', axis = 1)

# Sort the DataFrame based on the opening time in Melbourne time
df = df.sort_values(by='Melbourne Open Minutes')


# Drop the new column
df = df.drop(columns=['Melbourne Open Minutes'])

# Format the Melbourne Open and Melbourne Close columns
df['Melbourne Open'] = df['Melbourne Open'].apply(lambda x: x.strftime('%H:%M'))
df['Melbourne Close'] = df['Melbourne Close'].apply(lambda x: x.strftime('%H:%M'))

# Print the sorted DataFrame
df


Market UTC Open UTC Close Melbourne Open Melbourne Close  \
0   New York Stock Exchange    14:30     21:00          00:30           07:00   
2      Tokyo Stock Exchange    23:00     08:00          09:00           18:00   
5     Sydney Stock Exchange    00:00     06:00          10:00           16:00   
3   Shanghai Stock Exchange    01:30     07:00          11:30           17:00   
4  Hong Kong Stock Exchange    01:30     09:00          11:30           19:00   
1     London Stock Exchange    08:00     16:30          18:00           02:30   

   Status  
0  Closed  
2  Closed  
5  Closed  
3  Closed  
4  Closed  
1    Open